In [ ]:
import numpy as np
import pandas as pd
# After the k-means , this is our 2nd project on Unsupervised learning
# for the wrong movie names entered by the user , we'll install a package to find the closest match to that name
import difflib
# for converting the textual data into numbers that machine can understand , we'll use
from sklearn.feature_extraction.text import TfidfVectorizer
# for implementing our main cosine similarity algorithm , we'll use
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# data collection and preprocesing

In [ ]:
# importing the data movies into a pandas data frame
movies_data = pd.read_csv('/content/SpotifySongs.csv')

In [ ]:
movies_data = movies_data.assign(index=range(0, len(movies_data)))

In [ ]:
movies_data.head(100)

,SongName,ArtistName,Popularity,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,index
0,Saiyaan Ji,Yo Yo Honey Singh,62,0.896,0.678,0,-2.546,0,0.0815,0.01370,4.1e-05,0.11,0.604,130.024,205450,0
1,Falling,Trevor Daniel,84,0.785,0.431,10,-8.756,0,0.0364,0.12300,0.0,0.0887,0.236,127.085,159382,1
2,Shayad,Pritam,72,0.483,0.468,8,-7.642,1,0.0341,0.78500,0.0,0.152,0.319,135.864,247941,2
3,"Malang (Title Track) [From ""Malang - Unleash T...",Ved Sharma,68,0.384,0.728,7,-6.503,0,0.0623,0.32900,1.85e-05,0.329,0.311,156.396,287374,3
4,Genda Phool (feat. Payal Dev),Badshah,67,0.955,0.538,1,-8.394,1,0.0734,0.13200,0.00146,0.0704,0.837,116.977,170769,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Scholar's Mate,Carlos Rafael Rivera,95,0.563,0.664,9,-5.044,1,0.1540,0.33500,0.0,0.0849,0.688,166.928,178147,95
96,You're Gloating,Carlos Rafael Rivera,90,0.762,0.701,8,-3.541,1,0.0286,0.23500,0.000158,0.123,0.742,110.968,208867,96
97,Training With Mr. Schaibel,Carlos Rafael Rivera,89,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.0,0.0674,0.915,102.977,203064,97
98,good 4 u,Olivia Rodrigo,88,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.0,0.0906,0.695,109.997,164442,98


In [ ]:
movies_data.tail(100)

,SongName,ArtistName,Popularity,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,index
3631,When I Go,Keaton Simons,52,0.508,0.827,2,-8.612,1,0.0357,0.085,0.0134,0.0572,0.140,118.009,242239,3631
3632,Phantoms and Friends,Old Man Canyon,60,0.701,0.463,6,-7.212,0,0.0416,0.357,0.00306,0.073,0.352,139.941,313800,3632
3633,Waves,Dean Lewis,40,0.709,0.827,4,-7.304,0,0.0576,0.429,0.0205,0.985,0.850,105.300,277560,3633
3634,Wiser,Old Man Canyon,56,0.544,0.152,4,-17.995,0,0.0280,0.910,0.256,0.11,0.075,90.863,226184,3634
3635,In the Rain,Old Man Canyon,60,0.701,0.463,6,-7.212,0,0.0416,0.357,0.00306,0.073,0.352,139.941,313800,3635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3726,Back To Black,Amy Winehouse,79,0.766,0.706,7,-6.553,0,0.1600,0.223,0.00028,0.064,0.766,142.989,172598,3726
3727,Where Do We Go From Here,Charles Bradley,69,0.735,0.784,11,-7.128,0,0.0332,0.449,0.154,0.105,0.582,120.012,191000,3727
3728,Way down We Go - Stripped,KALEO,60,0.701,0.463,6,-7.212,0,0.0416,0.357,0.00306,0.073,0.352,139.941,313800,3728
3729,Let's Talk It Over,Lee Fields,54,0.595,0.628,1,-9.449,0,0.3170,0.183,0.315,0.122,0.170,102.029,230000,3729


In [ ]:
movies_data.shape

(3731, 16)

In [ ]:
# thats a big data bro , so I've to choose the important columns / relevant features

movies_data.isnull().sum()


SongName            0
ArtistName          0
Popularity          0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Duration_ms         0
index               0
dtype: int64

In [ ]:
# Selected features :
Selected_features = ['SongName','ArtistName','Popularity','Danceability','Energy','Mode','Instrumentalness','Liveness']
print(Selected_features)

['SongName', 'ArtistName', 'Popularity', 'Danceability', 'Energy', 'Mode', 'Instrumentalness', 'Liveness']


In [ ]:
# as we see above , there are a lot of missing values , so we've to replace them with null strings
for feature in Selected_features :
  movies_data[feature] = movies_data[feature].astype(str)

In [ ]:
# combining all the six selected features // +' '+movies_data['hitFlop']    ' '+movies_data['releaseYear']+ +' '+movies_data['sequel']
combined_features = movies_data['SongName']+' '+movies_data['ArtistName']+' '+movies_data['Popularity']+' '+movies_data['Danceability']+' '+movies_data['Energy']+' '+movies_data['Mode']+' '+movies_data['Instrumentalness']+' '+movies_data['Liveness']

In [ ]:
combined_features = combined_features.reset_index(drop=True)

In [ ]:
print(combined_features)

0       Saiyaan Ji Yo Yo Honey Singh 62 0.896 0.678 0 ...
1       Falling Trevor Daniel 84 0.785 0.431 0 0.0 0.0887
2                Shayad Pritam 72 0.483 0.468 1 0.0 0.152
3       Malang (Title Track) [From "Malang - Unleash T...
4       Genda Phool (feat. Payal Dev) Badshah 67 0.955...
                              ...                        
3726    Back To Black Amy Winehouse 79 0.766 0.706 0 0...
3727    Where Do We Go From Here Charles Bradley 69 0....
3728    Way down We Go - Stripped KALEO 60 0.701 0.463...
3729    Let's Talk It Over Lee Fields 54 0.595 0.628 0...
3730    The Gentle Rain - RJD2 Remix Astrud Gilberto 5...
Length: 3731, dtype: object


In [ ]:
Vectorizer = TfidfVectorizer()
features_vector = Vectorizer.fit_transform(combined_features)

In [ ]:
print(features_vector)

  (0, 426)	0.18578762048843278
  (0, 187)	0.12860236084410953
  (0, 520)	0.2396871179869471
  (0, 1021)	0.24393206158034625
  (0, 1258)	0.32560801821733126
  (0, 959)	0.19644223955744852
  (0, 3027)	0.2462458096630332
  (0, 2170)	0.32560801821733126
  (0, 3443)	0.5555922509306808
  (0, 2268)	0.32560801821733126
  (0, 2920)	0.32560801821733126
  (1, 344)	0.4160458065559377
  (1, 758)	0.3626930913005998
  (1, 1138)	0.42382091779576564
  (1, 1202)	0.2578813835199491
  (1, 1756)	0.33717489088031105
  (1, 3263)	0.40305286235144805
  (1, 1921)	0.4160458065559377
  (2, 472)	0.3970212913406607
  (2, 798)	0.4098197998447031
  (2, 815)	0.4263199470775047
  (2, 1069)	0.24990735517458693
  (2, 2779)	0.436777089940719
  (2, 2992)	0.48933139704481976
  (3, 651)	0.2068724383314297
  :	:
  (3728, 793)	0.288091548114064
  (3728, 1049)	0.2523260826833504
  (3729, 1944)	0.3783352626682613
  (3729, 2694)	0.3692773107219935
  (3729, 2397)	0.3146969446187079
  (3729, 439)	0.2726021784346069
  (3729, 873)	0.

In [ ]:
# to find the similarity score using cosine similarity algorithm
similarity = cosine_similarity(features_vector)

In [ ]:
print(similarity)

[[1.         0.         0.         ... 0.         0.         0.04325236]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.1314304 ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.04325236 0.         0.1314304  ... 0.         0.         1.        ]]


In [ ]:
# nxn matrix , as each movie is compared with each other movie just like a heat map ..

# Now getting input from the user :)
input_name = input('Enter the movie you want to see : ')

Enter the movie you want to see : Saiyyan Ji


In [ ]:
# making a list of all the movies in the dataset
list_of_all_movies = movies_data['SongName'].tolist()
print(list_of_all_movies)

['Saiyaan Ji', 'Falling', 'Shayad', 'Malang (Title Track) [From "Malang - Unleash The Madness"]', 'Genda Phool (feat. Payal Dev)', 'Humraah (From "Malang - Unleash The Madness")', 'Baila Conmigo (with Rauw Alejandro)', 'You (with Marshmello & Vance Joy)', 'Higher (feat. iann dior)', 'Element', 'HUMBLE.', 'Sucker for Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)', 'Dip (feat. Nicki Minaj) (feat. Nicki Minaj)', 'goosebumps', 'Walk It Talk It', 'King Kunta', 'Swang', "I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne)", 'Swang', 'Another Love', 'Another Love', 'People Help the People', 'unravel (acoustic version) - Acoustic Version', 'Isabella\'s Lullaby (from "The Promised Neverland")', 'asphyxia', 'Kokoronashi', 'Kokoronashi', 'unravel (acoustic version) - Acoustic Version', "Can't Hold Us - feat. Ray Dalton", 'Somebody That I Used To Know', 'River Flows In You', "Comptine d'un autre été, l'après-midi", 'You Raise Me Up', 'Hope'

In [ ]:
print(len(list_of_all_movies))

3731


In [ ]:
# ht

In [ ]:
# quite obvious .
# finding the close mataches to the movie entered by the user
close_matches = difflib.get_close_matches(input_name,list_of_all_movies)
print(close_matches)

['Saiyaan Ji']


In [ ]:
# finding the closest match /
closest_match = close_matches[0]
print(closest_match)

Saiyaan Ji


In [ ]:
# finding the index of the closest match
index_of_closest_match = movies_data[movies_data.SongName==closest_match]['index'].values[0]
print(index_of_closest_match)

0


In [ ]:
# finding all the similar movies to our closest match using similarity score

similarity_score = list(enumerate(similarity[index_of_closest_match]))
print(similarity_score)

[(0, 1.0), (1, 0.0), (2, 0.0), (3, 0.014926600330294689), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.02109667974740005), (9, 0.0), (10, 0.021396751220242954), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.02280486735296282), (20, 0.02280486735296282), (21, 0.019427772004305288), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.03752207273325347), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.05241216507244015), (39, 0.0), (40, 0.0), (41, 0.17414305941737265), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.022816423367105387), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.021288210875932866), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.0), (75

In [ ]:
len(similarity_score)

3731

In [ ]:
# obviously , the similarity score will compare our closest match to all the other movies(4803) and returns them in form of list in the same order as they were in dataset :)

In [ ]:
# now sorting the movies based on their similarity score value (highest to lowest)

In [ ]:
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(0, 1.0), (1680, 0.18067753144526624), (1828, 0.1785980749098027), (2486, 0.17503270569398938), (41, 0.17414305941737265), (954, 0.1713801894281422), (1008, 0.159548082412915), (2598, 0.1243845201397876), (3649, 0.11638488785421626), (228, 0.11415891453159831), (2987, 0.11032923501025764), (3409, 0.10373199154740015), (3447, 0.100521791134272), (1919, 0.09785267433575652), (447, 0.09766728217812201), (402, 0.09355706176174104), (3000, 0.09153232969142819), (2577, 0.09123236948565404), (82, 0.09023986706031066), (973, 0.09008530383338889), (901, 0.08963639315809291), (1368, 0.08872911138081699), (2869, 0.08793625419396872), (2323, 0.08713124832545979), (1543, 0.08655080542308428), (1544, 0.08655080542308428), (3171, 0.08638407109179716), (2632, 0.0863166972051463), (1780, 0.08604310306815856), (2870, 0.08518709502110511), (2690, 0.08463165518301412), (3470, 0.08387943695865513), (2599, 0.08311160457052798), (2213, 0.08259565428397606), (2428, 0.08252664774844998), (1258, 0.082183403290

In [ ]:
# now we'll write a code snippet to recommend the top 20 movies matches for the user's demand
# we used \n here to print in next line
print('Movies Recommendations for you are : \n')

i=1
for movie in sorted_similar_movies:
  index = movie[0]
  # now we'll be accessing the movie name by its' index
  title_from_index = movies_data[movies_data.index==index]['SongName'].values[0]
  if(i<=20):
    print(i,')',title_from_index)
    i+=1
  else :
    break

print('Thank You!')

Movies Recommendations for you are : 

1 ) Saiyaan Ji
2 ) Play Hard (feat. Ne-Yo & Akon) - New Edit
3 ) Famous (I'm The One) (feat. Yo Gotti & DeJ Loaf)
4 ) Famous (I'm The One) (feat. Yo Gotti & DeJ Loaf)
5 ) Famous (I'm The One) (feat. Yo Gotti & DeJ Loaf)
6 ) Famous (I'm The One) (feat. Yo Gotti & DeJ Loaf)
7 ) Love Again
8 ) Ae Watan (Male)
9 ) Bloodstream
10 ) Sygrayem (Let's Play)
11 ) Without Me
12 ) No Lie
13 ) exile (feat. Bon Iver)
14 ) Payphone
15 ) good 4 u
16 ) Righteous
17 ) Godzilla (feat. Juice WRLD)
18 ) Alone, Pt. II
19 ) Sideways
20 ) Shots
Thank You!


In [ ]:
# Now we'll put all the snippets to make a single cell Movie Recommender System :)

input_name = input('Enter the movie you want to see : ')
list_of_all_movies = movies_data['SongName'].tolist()
close_matches = difflib.get_close_matches(input_name,list_of_all_movies)
closest_match = close_matches[0]
index_of_closest_match = movies_data[movies_data.SongName==closest_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_closest_match]))
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse=True)
print('Movies Recommendations for you are : \n')

i=1
for movie in sorted_similar_movies:
  index = movie[0]
  # now we'll be accessing the movie name by its' index
  title_from_index = movies_data[movies_data.index==index]['SongName'].values[0]
  if(i<=20):
    print(i,')',title_from_index)
    i+=1
  else :
    break

print('Thank You!')

Enter the movie you want to see : Bewafa
Movies Recommendations for you are : 

1 ) Beware
2 ) Beware
3 ) Beware
4 ) Hope
5 ) Closer
6 ) Roses
7 ) Beware
8 ) Baarishein
9 ) Baarishein
10 ) good 4 u
11 ) good 4 u
12 ) Grenade
13 ) Dior
14 ) Beware
15 ) Run
16 ) Good Years
17 ) Lily
18 ) Higher Power
19 ) working
20 ) Love The Way You Lie
Thank You!


In [ ]:
print('20+LPAPls....')

20+LPAPls....
